In [1]:
import markdown
import faiss

In [2]:
from utils import embed
from config import CONFIG

# embed and save embeddings
vector_store = embed(CONFIG["document_folder"])

Vector store save to faiss_index


In [3]:
from utils import load

vector_store = load()

Vector store loaded


In [4]:
from langchain.text_splitter import MarkdownTextSplitter
import markdown

file = r"C:\Users\shewe\OneDrive\Documents\Obsidian Vault\Vector Database.md"
f = open(file, 'r')
markdown_text = f.read()
htmlmarkdown=markdown.markdown( f.read() )

splitter = MarkdownTextSplitter(chunk_size=200)
chunks = splitter.split_text(htmlmarkdown)


In [5]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_text)
md_header_splits[0].metadata


{'Header 1': 'Vector Database'}

[('#', 'Header 1'), ('##', 'Header 2'), ('###', 'Header 3')]

In [12]:
## recursive text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", "#", "##", "###"]
)

doc = Document(page_content=markdown_text)

# Split it
chunks = text_splitter.split_documents([doc, doc])

# See the result
for i, chunk in enumerate(chunks[:3]):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
page_content='# Vector Database  
#tag: #concept

Also known as **vector DB**, this refers to a specialized database designed to store and search high-dimensional vectors — usually produced by an embedding model.

## How It Works

1. **Input Data**  
   You start with raw data such as text (essays, code, etc.).

2. **Embedding**  
   An **embedding model** (e.g., OpenAI, Cohere, or sentence-transformers) converts the input into a high-dimensional vector (e.g., 384 or 768 dimensions).'

Chunk 2:
page_content='3. **Storage**  
   These vectors are stored in a vector database like **FAISS**, **Pinecone**, or **Weaviate**.

4. **Querying**  
   When a user submits a query (e.g., a question), the **same embedding model** is used to convert the query into a vector.

5. **Similarity Search**  
   The vector DB uses a similarity algorithm (like **cosine similarity** or **approximate nearest neighbors**) to find the most similar vectors to the query.

## Metadata Storage and Filtering'

In [13]:
chunks

[Document(metadata={}, page_content='# Vector Database  \n#tag: #concept\n\nAlso known as **vector DB**, this refers to a specialized database designed to store and search high-dimensional vectors — usually produced by an embedding model.\n\n## How It Works\n\n1. **Input Data**  \n   You start with raw data such as text (essays, code, etc.).\n\n2. **Embedding**  \n   An **embedding model** (e.g., OpenAI, Cohere, or sentence-transformers) converts the input into a high-dimensional vector (e.g., 384 or 768 dimensions).'),
 Document(metadata={}, page_content='3. **Storage**  \n   These vectors are stored in a vector database like **FAISS**, **Pinecone**, or **Weaviate**.\n\n4. **Querying**  \n   When a user submits a query (e.g., a question), the **same embedding model** is used to convert the query into a vector.\n\n5. **Similarity Search**  \n   The vector DB uses a similarity algorithm (like **cosine similarity** or **approximate nearest neighbors**) to find the most similar vectors to

In [ ]:
# 3. Search for similar items
query = "what is mode seeking loss"
query_vector = vector_store.similarity_search(query, k=2)

context = "\n\n".join([doc.page_content for doc in query_vector])
prompt = f"Answer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"

from langchain_community.llms import Ollama 
llm = Ollama(model="llama3.2")
response = llm.predict(prompt)

print(response)


C:\Users\shewe\AppData\Local\Temp\ipykernel_17780\3181714049.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")
C:\Users\shewe\AppData\Local\Temp\ipykernel_17780\3181714049.py:10: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


Mode Seeking Loss is a loss function introduced in DFMGAN, which aims to maximize the generation of defects with different latent spaces $z_{defect}$. It is considered an intrinsic motivation technique, meaning it encourages the generator to produce better results without relying on extrinsic rewards or penalties.


In [11]:
query = "gan"
query_vector = vector_store.similarity_search(query, k=2)

for i, doc in enumerate(query_vector):
    print(f"Document {i+1}:")
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

Document 1:
Metadata: {'Title': 'html'}
--------------------------------------------------
Document 2:
Metadata: {'Title': 'html'}
--------------------------------------------------


In [5]:
results = vector_store.as_retriever().get_relevant_documents("gan")

C:\Users\shewe\AppData\Local\Temp\ipykernel_16168\3622681122.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = vector_store.as_retriever().get_relevant_documents("gan")


In [10]:
for doc in results:
    print(doc.metadata['Title'])

html
html
html
html
